# Exemplo

## Descrição do exemplo

Exemplo retirado do livro xx.... estimação dos parâmetros E, T de ....

Modelo a ser utilizado nos cálculos:

$y_1 = e^{-(k_o\cdot 10^{17})\cdot t \cdot e^{(-\frac{E}{T})}}$

## Primeiro passo:

Importação de pacotes básicos

In [10]:
# Importando as funções
from MT_PEU import EstimacaoNaoLinear
from MT_PEU_Linear import EstimacaoLinear
# Importantando exp
from numpy import exp

## Segundo passo: Escrever o modelo

In [11]:
def modelo(param,x,args):

    tempo = x[:,0:1] # primeira coluna do vetor de dados
    T     = x[:,1:2] # segunda coluna do vetor de dados

    ko = param[0] # primeiro parâmetro
    E  = param[1] # segundo parâmetro

    y1 = exp(-ko*tempo*exp(-E*(1/T-1./630.))) # cálculo de y - vetorizado

    return y1


## Terceiro passo: inclusão de dados experimentais

In [12]:
# Tempo

x1 = [120.0,60.0,60.0,120.0,120.0,60.0,60.0,30.0,15.0,60.0,
45.1,90.0,150.0,60.0,60.0,60.0,30.0,90.0,150.0,90.4,120.0,
60.0,60.0,60.0,60.0,60.0,60.0,30.0,45.1,30.0,30.0,45.0,15.0,30.0,90.0,25.0,
60.1,60.0,30.0,30.0,60.0]

# Incerteza associada a cada ponto
ux1 = [1]*41

# Temperatura
x2 = [600.0,600.0,612.0,612.0,612.0,612.0,620.0,620.0,620.0,
620.0,620.0,620.0,620.0,620.0,620.0,620.0,620.0,620.0,620.0,620.0,620.0,
620.0,620.0,620.0,620.0,620.0,620.0,631.0,631.0,631.0,631.0,631.0,639.0,639.0,
639.0,639.0,639.0,639.0,639.0,639.0,639.0]

# Incerteza associada a cada ponto
ux2 = [1]*41

# Variável dependente

y = [0.9,0.949,0.886,0.785,0.791,0.890,0.787,0.877,0.938,
0.782,0.827,0.696,0.582,0.795,0.800,0.790,0.883,0.712,0.576,0.715,0.673,
0.802,0.802,0.804,0.794,0.804,0.799,0.764,0.688,0.717,0.802,0.695,0.808,
0.655,0.309,0.689,0.437,0.425,0.638,.659,0.449]

uy = [1]*41

## Terceiro passo: Iniciar a função para estimação

In [13]:
Estime = EstimacaoNaoLinear(modelo, ['y'], ['x1','x2'], ['ko','E'], PA=0.95, projeto='Projeto')
Estime.setDados(0,(x1,ux1),(x2,ux2))
Estime.setDados(1,(y,uy))
Estime.setConjunto(tipo='estimacao')
Estime.otimiza(estimativa_inicial= [0.005, 20000.000],algoritmo='Powell')
Estime.incertezaParametros(delta=1e-5,metodoIncerteza='SensibilidadeModelo',preencherregiao=False)
Estime.predicao()
Estime.analiseResiduos()
etapas = ['otimizacao','grandezas-entrada', 'predicao','grandezas-calculadas','analiseResiduos', 'regiaoAbrangencia']
etapas = ['grandezas-entrada']
Estime.graficos(etapas)
Estime.relatorio(export_y=True,export_cov_y=True)

<Figure size 432x288 with 0 Axes>